In [68]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
%autoreload 2

In [94]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime

''' Sn properties - Masses, Ratios, Cup Configurations, ... '''
from sn_config import *
from collections import Counter
import pandas as pd
import numpy as np

'''Classes for Reading in the Data and applying Internal Normalisation'''
from nu_data_reduction import NU_data_read, int_norm, evaluation, Neptune_data_read
from parameter import *
from dspike_formulas import *



In [95]:
path = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2017-04-27/"
files_start = 7208
files_end = 7209

In [96]:
spikeSn_ls = [["117"],["118", "122", "124"]]
spike_dict_117_122 = {"112" : 0.0001458, "114" : 0.0001458, "115": 0.0001458,"116" : 0.0423854, "117": 0.5016273,
              "118": 0.0052716, "119" : 0.0048706, "120" : 0.0175002, "122" : 0.4221922, "124" : 0.0076096} # assumed DS compositon

meas_dict = {"112" : 0.029823, "114" : 0.020190, "115" : 0.010361, "116" : 0.4460, "117" : 0.235318,
             "118" : 0.742944, "119" : 0.263474, "122" : 0.142095, "124" : 0.177579} # mean of 200ppb NIST Std

In [97]:
# cup configuration
#cup_config = cycle_Sb
Sn_isotopes = ["117", "118", "119", "120", "122", "124"]
cup_config = cycles2

# Mass Range of cup configuration
mass_range = cycles2_mass_range

# Isotopes used for Interference correction
corr_isotopes_2 = {"Te" : "126"}
#corr_isotopes_2 = {}
#corr_isotopes_Sb = {"Te": "125"}

denom_isotope = "117"
Sn_monitor = ["126"]

#def eval_iso_list(isotopes_list, denom_isotope, monitor_iso):
#    isotope_den = denom_isotope
#    isotopes_list.remove(isotope_den)
#    for i in range(len(monitor_iso)):
#        isotopes_list.append(monitor_iso[i])
#    isotopes_list.sort()
#   return isotopes_list

#isotope_ls = eval_iso_list(Sn_isotopes, denom_isotope, Sn_monitor)

#data_sample_column = [(i + "/" + denom_isotope) for i in isotope_ls]    
isotopes = [["118", "122", "124"]]

#number of iterations for beta
iter_beta = 10

# Interference_corr on the denominator isotope
isotope_denom_corr = False
# background correction
blk_corr = False

In [46]:
isotopes

[['118', '122', '124']]

In [25]:
columns_1 = ["cycle", "sample", "date", "H8 (2)", "H7 (2)", "H6 (2)", "H5 (2)", "H4 (2)", "H3 (2)", "H2 (2)", "H1 (2)", "Ax (2)", "L1 (2)", "L2 (2)", "L3 (2)", "L4 (2)"]
df_all = pd.DataFrame()
blk_ls = []
sample_ls = []
files_1 = range(files_start, files_end, 1)
#files_1 = range(5552,5623,1) + range(5632,5729, 1)

for sample in files_1:
    
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    df_zero = df.data_zero_corr(sample)
    df_zero_1 = pd.DataFrame(df_zero["cycle2"])
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    df_zero_1["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    df_zero_1["sample"] = sample_name
    df_zero_1["cycle"] = df_zero_1.index.values
    df_zero_1.index = df_zero_1["date"]
    df_zero_1 = df_zero_1[columns_1]
    
    if sample_name == "blank sol" or sample_name == "wash" or sample_name == "wash clean" or sample_name == "Teflon blk":
        blk_ls.append(sample)
    elif sample_name == "SQ" or sample_name == "Teflon blk" or sample_name == "4ml HDPE blank":
        None
    else:
        sample_ls.append(sample)

df_zero_1.set_index("date", inplace = True)

#for x in [123]:
#    sample_ls.remove(x)
#blk_ls = blk_ls[0::5]
#blk_ls.remove(5234)

In [9]:
df_zero_1

,cycle,sample,H8 (2),H7 (2),H6 (2),H5 (2),H4 (2),H3 (2),H2 (2),H1 (2),Ax (2),L1 (2),L2 (2),L3 (2),L4 (2)
date,,,,,,,,,,,,,,,
2017-04-27 16:22:00,1,NIST 200ppb,0.008605,0.010430,0.014287,0.000370,0.000168,2.580615,0.000397,2.011458,0.000414,13.786305,3.583036,9.967819,3.114180
2017-04-27 16:22:00,2,NIST 200ppb,0.008640,0.010470,0.014337,0.000386,0.000190,2.750147,0.000441,2.143773,0.000459,14.691961,3.818205,10.622796,3.318864
2017-04-27 16:22:00,3,NIST 200ppb,0.008610,0.010468,0.014349,0.000381,0.000189,2.739335,0.000425,2.134998,0.000423,14.633444,3.803532,10.579996,3.305644
2017-04-27 16:22:00,4,NIST 200ppb,0.008576,0.010459,0.014320,0.000369,0.000184,2.703765,0.000457,2.107219,0.000448,14.443088,3.753818,10.442598,3.262708
2017-04-27 16:22:00,5,NIST 200ppb,0.008652,0.010467,0.014356,0.000378,0.000190,2.649831,0.000431,2.065890,0.000417,14.156836,3.679239,10.236512,3.198021
2017-04-27 16:22:00,6,NIST 200ppb,0.008613,0.010464,0.014342,0.000383,0.000201,2.713914,0.000429,2.115925,0.000460,14.499475,3.768370,10.484205,3.275485
2017-04-27 16:22:00,7,NIST 200ppb,0.008600,0.010439,0.014319,0.000367,0.000196,2.680265,0.000443,2.089256,0.000420,14.318691,3.721508,10.352921,3.234682
2017-04-27 16:22:00,8,NIST 200ppb,0.008603,0.010452,0.014358,0.000354,0.000194,2.643091,0.000412,2.060203,0.000437,14.120098,3.669981,10.209566,3.190018
2017-04-27 16:22:00,9,NIST 200ppb,0.008606,0.010461,0.014353,0.000390,0.000188,2.762329,0.000436,2.153214,0.000443,14.756535,3.835296,10.670026,3.333708


In [8]:
# For one datapoint
files_1 = 7208

df = NU_data_read(path, sample, cup_config)
cycles = range(1, len(df.data_read(sample).index)+1)

df_zero = df.data_zero_corr(sample)
new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_2 , denom_corr_ratio)
#new_corr.line2_corr(df_zero, "119")

data_sample = new_corr.raw_ratios(denom_isotope)
data_sample = pd.DataFrame.from_dict(data_sample, orient = 'index')
df_zero

NameError: name 'sample' is not defined

In [85]:
path = "/Volumes/friebelm/PhD/Double Spike/Planning Sn 117-122/real data -intern norm/"
data_sample = pd.read_csv(str(path + "sn_fake_spike_mix.csv"))
data_sample

,118,122,124
0,0.611626,0.868462,0.166942
1,0.611635,0.868524,0.166965
2,0.611610,0.868468,0.166942
3,0.611635,0.868500,0.166952
4,0.611632,0.868459,0.166941
5,0.611592,0.868393,0.166938
6,0.611631,0.868470,0.166954
7,0.611604,0.868456,0.166944
8,0.611619,0.868482,0.166959
9,0.611635,0.868453,0.166939


In [86]:
#Spike Calculation uncorrected
data_spike_obj = calc_dspike_sample(Sn_meas_obj, data_sample, spike_obj, Sn_mass_obj, spikeSn_ls , denom_isotope)
data_spike_calc = data_spike_obj.dspike_corr(3, 6, -0.1, -2.2, 'x')
beta = data_spike_calc["frac_ins_x2.5"]

In [87]:
# turned hypothetical sample-spike mix in signals and loaded as new dataframe for interference corr
data_sample_2 = pd.read_csv(str(path + "sn_fake_spike_mix_signals.csv"))
data_spike = {}
data_spike["cycle1"] =  data_sample_2.to_dict()
data_sample_2

,118,122,124,117,125,126,129
0,1.904714,2.704546,0.519888,3.114180,0.000168,0.000370,0.010430
1,2.029933,2.882513,0.554133,3.318864,0.000190,0.000386,0.010470
2,2.021766,2.870847,0.551852,3.305644,0.000189,0.000381,0.010468
3,1.995586,2.833662,0.544716,3.262708,0.000184,0.000369,0.010459
4,1.956013,2.777350,0.533880,3.198021,0.000190,0.000378,0.010467
5,2.003260,2.844407,0.546803,3.275485,0.000201,0.000383,0.010464
6,1.978430,2.809226,0.540043,3.234682,0.000196,0.000367,0.010439
7,1.951028,2.770391,0.532555,3.190018,0.000194,0.000354,0.010452
8,2.038959,2.895265,0.556592,3.333708,0.000188,0.000390,0.010461
9,2.044290,2.902661,0.557967,3.342334,0.000205,0.000401,0.010485


In [98]:
#Interference correction based on beta from first DS-Inversion
cup_config_x = {"cycle1" : {"129" : "129", "126" : "126", "125" : "125", "124" : "124", "122" : "122", "118" : "118", "117" : "117"}}
mass_range_x = Isotopes_Mass_Range()
mass_range_x.add_mass_range_dict({"117" : ["Sn"], "118" : ["Sn"], "122" : ["Sn", "Te"], "124" : ["Sn", "Te", "Xe"], "125" : ["Te"], "126": ["Te", "Xe"], "129" : ["Xe"]})
data_corr = {}
data_corr_raw = {}
cycle_no = 20
cycles = range(0, len(data_sample.index))

for n in cycles:
    beta_x = beta[n]
    spike_corr = int_norm(data_spike, n, cup_config_x, database, mass_range_x, corr_isotopes_2)
    
    data_corr[n] = {}
    data_corr_raw[n] = {}
    for isotope in isotopes[0]:
        # interference corrected ratios
        data_corr[n][isotope] = spike_corr.interference_corr_ratio("Sn", isotope , "117", beta[n])
        #use r = R(m2/m1)^-beta -> raw ratios for next double spike inversion
        mass_isotope_nom = spike_corr.database["Sn"]["Masses"].get_Isotope_mass(isotope) #e.g. M(116Sn)
        mass_isotope_denom = spike_corr.database["Sn"]["Masses"].get_Isotope_mass("117")# e.g. M(120Sn)
        data_corr_raw[n][isotope] = data_corr[n][isotope] * (mass_isotope_nom/mass_isotope_denom)**(-beta[n])
        

In [99]:
pd.DataFrame.from_dict(data_corr_raw, orient = 'index')

,122,124,118
0,0.868447,0.166914,0.611626
1,0.868510,0.166937,0.611635
2,0.868454,0.166914,0.611610
3,0.868486,0.166925,0.611635
4,0.868444,0.166912,0.611632
5,0.868378,0.166910,0.611592
6,0.868456,0.166927,0.611631
7,0.868442,0.166917,0.611604
8,0.868467,0.166931,0.611619
9,0.868438,0.166910,0.611636


In [79]:
element_denom = "Sn"
isotope_denom = "117"
beta_x = beta[0]

corr_dict = {}
counter = 0
for level in mass_range_x.get_order_of_corr(corr_isotopes_2):
    if counter == 0:
        for isotope in level:
            element_nom = mass_range_x.get_isotopes(isotope)[0]
            corr_dict[isotope] = spike_corr.int_norm(element_nom, element_denom, isotope, isotope_denom, beta)
        counter += 1
    elif counter > 0:
        for isotope in level:
            corr_isotopes = list(mass_range_x.get_graph_of_corr(corr_isotopes_2)[isotope])
            if (len(corr_isotopes) > 1):
                for mass_no in corr_isotopes:
                    if mass_no in mass_range_x.get_order_of_corr(corr_isotopes_2)[counter-1]:
                        corr_isotope = mass_no
            else:
                corr_isotope = corr_isotopes[0]

            element_nom = set(mass_range_x.get_isotopes(isotope))
            element_nom = element_nom.difference(mass_range_x.get_isotopes(corr_isotope))
            element_nom = list(element_nom)[0]
            element_corr = set(mass_range_x.get_isotopes(corr_isotope))
            element_corr = element_corr.intersection(mass_range_x.get_isotopes(isotope))
            element_corr = list(element_corr)
            
            if (len(element_corr) > 1):
                print element_corr
                corr_dict[isotope] = spike_corr.int_norm(element_nom, element_denom, isotope, isotope_denom, beta)
                for element in element_corr:
                    corr_isotope_x = corr_isotopes_2[element]
                    print isotope
                    print corr_isotope_x
                    true_ratio_corr = database[element]["Ratios"].get_ratio(isotope, corr_isotope_x)
                    print true_ratio_corr
                    corr_dict[isotope] -= corr_dict[corr_isotope_x] * true_ratio_corr
                    print corr_dict[corr_isotope_x]
                    print corr_dict[isotope]
            else:
                print isotope
                corr_dict[isotope] = spike_corr.int_norm(element_nom, element_denom, isotope, isotope_denom, beta)
                print corr_isotope
                true_ratio_corr = database[element_corr[0]]["Ratios"].get_ratio(isotope, corr_isotope)
                corr_dict[isotope] -= corr_dict[corr_isotope] * true_ratio_corr
                
            
        counter += 1

122
125
124
125
126
129


In [80]:
mass_range_x.get_graph_of_corr(corr_isotopes_2)

{'117': set(),
 '118': set(),
 '122': {'125'},
 '124': {'125', '129'},
 '125': set(),
 '126': {'129'},
 '129': set()}

In [81]:
mass_range_x.get_order_of_corr(corr_isotopes_2)

[{'117', '118', '125', '129'}, {'122', '124', '126'}]

In [67]:
graph = {}
for mass_no in mass_range_x.isotopes_mass_range:
    print mass_no
    isotopes = mass_range_x.isotopes_mass_range[mass_no]
    graph[mass_no] = set()
    for isotope in isotopes:
        print isotope
        if isotope in corr_isotopes_2 and set(isotopes).issubset(corr_isotopes_2) == False:
            if isotope == mass_range_x.isotopes_mass_range[mass_no][0]:
                None
            else:
                print mass_no
                graph[mass_no].add(corr_isotopes_2[isotope])
            #print isotope
            #print set(isotopes).issubset(corr_isotopes_2)
            #print corr_isotopes_2[isotope]
        elif isotope not in corr_isotopes_2:
            None
        elif len(mass_range_x.isotopes_mass_range[corr_isotopes_2[isotope]]) == 1 and len(mass_range_x.isotopes_mass_range[mass_no]) > 1:
            if isotope == mass_range_x.isotopes_mass_range[mass_no][0]:
                None
            else:
                print mass_no
                graph[mass_no].add(corr_isotopes_2[isotope])

graph

122
Sn
Te
122
117
Sn
124
Sn
Te
124
Xe
124
125
Te
126
Te
Xe
126
129
Xe
118
Sn


{'117': set(),
 '118': set(),
 '122': {'125'},
 '124': {'125', '129'},
 '125': set(),
 '126': {'129'},
 '129': set()}

In [90]:
data_corr_raw = pd.DataFrame.from_dict(data_corr_raw, orient = 'index')
#pd.DataFrame.from_dict(data_corr, orient = 'index')
data_corr_raw

,122,124,118
0,0.868448,0.166905,0.611626
1,0.868511,0.166929,0.611635
2,0.868455,0.166906,0.611610
3,0.868487,0.166917,0.611635
4,0.868445,0.166904,0.611632
5,0.868379,0.166902,0.611592
6,0.868457,0.166918,0.611631
7,0.868444,0.166909,0.611604
8,0.868469,0.166923,0.611619
9,0.868440,0.166902,0.611636


In [91]:
data_spike_obj = calc_dspike_sample(Sn_meas_obj, data_corr_raw, spike_obj, Sn_mass_obj, spikeSn_ls , denom_isotope)
data_spike_calc_2 = data_spike_obj.dspike_corr(3, 6, -0.1, -2.2, 'x')
data_spike_calc_2["frac_ins_x2.5"]

0    -2.200630
1    -2.202190
2    -2.200687
3    -2.201707
4    -2.200622
5    -2.198177
6    -2.200656
7    -2.200200
8    -2.200817
9    -2.200494
10   -2.199512
11   -2.199293
12   -2.200679
13   -2.202520
14   -2.199945
15   -2.200194
16   -2.201658
17   -2.198440
18   -2.200815
19   -2.199085
Name: frac_ins_x2.5, dtype: float64

In [22]:
data_spike_calc_2.to_csv(path+"test_2.csv")

In [47]:
files_1 = range(files_start, files_end, 1)
df_raw_signals = pd.DataFrame()
df_internal_norm = pd.DataFrame()

for sample in sample_ls:
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    # arbitrary blank positions
    #blk1 = [item for item in blk_ls if item < sample]
    #blk2 = [item for item in blk_ls if item > sample]
    #blk1 = blk1[-1]
    #blk2 = blk2[0]
    #blk_corr_sample = True
    
    # blank measurement surrounds sample
    #if ((sample-1) in blk_ls) and ((sample+1) in blk_ls):
    #    blk1 = sample-1
    #    blk2 = sample+1
    #    blk_corr_sample = True
    #else:
    #    blk_corr_sample = False
    
    if (blk_corr == True) and (blk_corr_sample == True):
        blk_1 = NU_data_read(path, blk1, cup_config)
        blk_2 = NU_data_read(path, blk2, cup_config)
        df_zero = df.data_zero_corr(sample)
        df_bgd_1 = blk_1.data_zero_corr(blk1)
        df_bgd_2 = blk_2.data_zero_corr(blk2)   
        
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_2 , denom_corr_ratio)
        new_corr.line2_corr(df_zero, "119")
        df_bgd_1 = new_corr.mad_outlier_rejection_dict(df_bgd_1)
        df_bgd_2 = new_corr.mad_outlier_rejection_dict(df_bgd_2)
        new_corr.data_bgd_corr(df_bgd_1, df_bgd_2)
    else:
        df_zero = df.data_zero_corr(sample)
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_2 , denom_corr_ratio)
        new_corr.line2_corr(df_zero, "119")
    
    data_sample_raw = new_corr.raw_signals(denom_isotope)
    data_sample_raw = pd.DataFrame.from_dict(data_sample_raw, orient = 'index')
    data_sample_raw = new_corr.mad_outlier_rejection(data_sample_raw)
    
    data_sample = new_corr.raw_ratios(denom_isotope)
    data_sample = pd.DataFrame.from_dict(data_sample, orient = 'index')
    data_spike_obj = calc_dspike_sample(Sn_meas_obj, data_sample, spike_obj, Sn_mass_obj, spikeSn_ls , denom_isotope)
    data_spike_calc = data_spike_obj.dspike_corr(3, 6, 0.0069, -2.24, 'z')
    data_spike_calc = new_corr.mad_outlier_rejection(data_spike_calc)
    
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    data_spike_calc["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_spike_calc["sample"] = sample_name
    data_spike_calc["cycle"] = data_sample.index.values
    
    #data_sample_raw["cycle"] = data_sample_raw.index.values
    #data_sample = data_sample.merge(data_sample_raw)
    
    #df_internal_norm = df_internal_norm.append(data_sample)

KeyError: 0

In [6]:
spikeCd_ls = [["111"],["113", "112", "114"]]

In [7]:
Cd_masses ={"110" : 109.903007, "111" : 110.904182, "112" : 111.9027577,
            "113" : 112.9044014, "114" : 113.9033586, "116" : 115.904756}

Cd_spike = {"112" : 0.025899, "113" : 0.470870, "114" : 0.020021}
Cd_spike_denom = "111"

Cd_std = {"112" : 1.878410, "113" : 0.950024, "114" : 2.227540}
Cd_std_denom = "111"

In [8]:
# Masses object used for calc
Cd_mass_obj = load_mass_dict(Cd_masses)

# Abundance object used for calc
Cd_std_obj = load_ratio_dict(Cd_std, Cd_std_denom)
Cd_spike_obj = load_ratio_dict(Cd_spike, Cd_spike_denom)

In [9]:
sample_1567 = calc_dspike_sample(Cd_std_obj,df_new,Cd_spike_obj,Cd_mass_obj,spikeCd_ls ,"111")
sample_1567_res = sample_1567.dspike_corr(3, 6, 0.0069, -2.24, 'z')
print sample_1567_res["Nr2:z"].mean()
print ((sample_1567_res["Nr2:z"].std()/np.sqrt(60))/sample_1567_res["Nr2:z"].mean())*10**6
print 2.227540 * (113.9033586/110.904182)** sample_1567_res["frac_nat_z2"].mean()
#sim1_mix.to_csv(path+'/Spike_Sim/'+"sim_117_122,120 d=117, q=0.48, p=0.36.csv")

2.2246218983
82.0132881511
2.22462145682


In [10]:
sample_1567_res

,n0:x,n0:y,n0:z,N0:x,N0:y,N0:z,a_nat0,b_nat0,c_nat0,m0.0:x,...,e_nat2,f_nat2,g_nat2,a_ins2,b_ins2,c_ins2,Nr2:x,Nr2:y,Nr2:z,frac_nat_z2
0,0.950024,1.87841,2.22754,0.950141,1.878526,2.22795,3.118599,0.385007,-1.458405,0.568895,...,-1.097095,3.528774,-4.400945,0.728247,1.002542,-0.348854,0.949261,1.877653,2.224869,-0.044968
1,0.950024,1.87841,2.22754,0.950141,1.878526,2.22795,3.118599,0.385007,-1.458405,0.568993,...,-1.096460,3.527418,-4.398398,0.727553,1.002015,-0.348513,0.948525,1.876923,2.222294,-0.088360
2,0.950024,1.87841,2.22754,0.950141,1.878526,2.22795,3.118599,0.385007,-1.458405,0.568787,...,-1.096962,3.528490,-4.400411,0.727736,1.002526,-0.348612,0.949106,1.877500,2.224329,-0.054064
3,0.950024,1.87841,2.22754,0.950141,1.878526,2.22795,3.118599,0.385007,-1.458405,0.568861,...,-1.097353,3.529325,-4.401979,0.727926,1.002912,-0.348712,0.949560,1.877949,2.225915,-0.027355
4,0.950024,1.87841,2.22754,0.950141,1.878526,2.22795,3.118599,0.385007,-1.458405,0.568773,...,-1.097449,3.529529,-4.402362,0.728077,1.002979,-0.348785,0.949670,1.878059,2.226303,-0.020824
5,0.950024,1.87841,2.22754,0.950141,1.878526,2.22795,3.118599,0.385007,-1.458405,0.568886,...,-1.096882,3.528320,-4.400091,0.727803,1.002420,-0.348641,0.949014,1.877408,2.224005,-0.059512
6,0.950024,1.87841,2.22754,0.950141,1.878526,2.22795,3.118599,0.385007,-1.458405,0.568953,...,-1.097162,3.528917,-4.401212,0.727907,1.002704,-0.348698,0.949338,1.877730,2.225139,-0.040411
7,0.950024,1.87841,2.22754,0.950141,1.878526,2.22795,3.118599,0.385007,-1.458405,0.568865,...,-1.096381,3.527249,-4.398080,0.727437,1.001957,-0.348457,0.948433,1.876832,2.221973,-0.093779
8,0.950024,1.87841,2.22754,0.950141,1.878526,2.22795,3.118599,0.385007,-1.458405,0.568963,...,-1.097087,3.528757,-4.400913,0.727976,1.002603,-0.348727,0.949252,1.877644,2.224837,-0.045502
9,0.950024,1.87841,2.22754,0.950141,1.878526,2.22795,3.118599,0.385007,-1.458405,0.568902,...,-1.097153,3.528897,-4.401175,0.727960,1.002680,-0.348722,0.949327,1.877719,2.225102,-0.041035
